In [1]:
"""
Run MeasureModuleEfficiency.py in parallel using concurrent.futures

Be really careful with this. 
"""

'\nRun MeasureModuleEfficiency.py in parallel using concurrent.futures\n\nBe really careful with this. \n'

In [2]:
# import sys
# from datetime import datetime
import pandas as pd
from concurrent.futures import ProcessPoolExecutor, as_completed
from mu2etools import read_data as rd 
import numpy as np
import subprocess

In [3]:
# # Generate a timestamped log filename
# timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
# logName = f"../Logs/output_{timestamp}.log"
# # Original standard output state
# print("Log file is", logName)

In [26]:
# Multiprocess(processFunction, fileList_, recon, particles_, PEs_, layers_, triggerModes_, quiet, resub=False)
def Multiprocess(processFunction, fileList_, recon, particles_, PEs_, layers_, triggerModes_, quiet, resub=False, max_workers=16):
    print("\n---> Starting multiprocessing...\n")

    print(layers_)
    
    # global stop_flag
    
    completedFiles = 0
    totalFiles = len(fileList_)
    
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        # Prepare a list of futures and map them to file names
        futures_ = {}

        if not resub: 
            futures_ = {executor.submit(processFunction, fileName, recon, particles_, PEs_, layers_, triggerModes_, quiet): fileName for fileName in fileList_}
        else:
            failedJobsFile = "../Txt/MDC2020ae/FailedJobs/failures.csv"
            failedJobs_ = pd.read_csv(failedJobsFile)
            failedFileList = [] 
            for fileName in fileList_:
                finTag = fileName.split('.')[-2] 
                if finTag not in list(failedJobs_["Tag"]):
                    continue
                else:
                    thisFailedJob = failedJobs_[failedJobs_["Tag"] == finTag]
                    PEs_ = list(thisFailedJob["PEs"]) 
                    particles_ = list(thisFailedJob["Particle"])  
                    layers_ = list(thisFailedJob["Layer"])  
                    
                future = executor.submit(processFunction, fileName, recon, particles_, PEs_, layers_, triggerModes_, quiet)
                futures_[future] = fileName    
                
        # try:
            
        # Process results as they complete
        for future in as_completed(futures_):
            fileName = futures_[future]  # Get the file name associated with this future
            try:
                future.result()  # Retrieve the result
                completedFiles += 1
                percentComplete = (completedFiles / totalFiles) * 100
                print(f'\n---> {fileName} processed successfully! ({percentComplete:.1f}% complete)')
            except Exception as exc:
                print(f'\n---> {fileName} encountered an exception: {exc}')

        # except KeyboardInterrupt:
        #     print("---> KeyboardInterrupt received, stopping...")
        #     stop_flag.set()  # Signal all processes to stop
        #     executor.shutdown(wait=True)
        # finally:
        #     stop_flag.set()  # Ensure all processes are signaled to stop
                
    print("\n---> Multiprocessing completed!")
    return

In [25]:
def processFunction(fileName, recon, particles_, PEs_, layers_, triggerModes_, quiet):
    try:
        # Convert list arguments to string representations for passing via the command line
        particles_str = ','.join(particles_)
        PEs_str = ','.join(map(str, PEs_))
        layers_str = ','.join(map(str, layers_))
        triggerModes_str = ','.join(triggerModes_)

        # Path to the Python executable in your Conda environment
        python_executable = "/home/sgrant/.conda/envs/mu2e_env/bin/python" 
        
        # Call the script MeasureModuleEfficiency.py using subprocess
        subprocess.run(
            [
                python_executable, '/home/sgrant/CRVSim/Analyses/PyMacros/MeasureModuleEfficiency.py',
                fileName, recon, particles_str, PEs_str, layers_str, triggerModes_str, str(quiet)
            ],
            check=True
        )
    except subprocess.CalledProcessError as exc:
        print(f'---> Exception!\n{exc}')
    return

In [6]:
#########################################################
defname = "nts.sgrant.CosmicCRYExtractedCatTriggered.MDC2020ae_best_v1_3.root"
recon = "MDC2020ae"
particles_ = ["all", "muons", "non_muons"]
layers_ = [2, 3] 
PEs_ = np.arange(10, 135, 5)
triggerModes_ = ["crv_trigger", "trk_trigger", "trk_crv_trigger", "trk_crv2_trigger", "trk_crv3_trigger"]
quiet = True

In [7]:
import itertools 
combinations = list(itertools.product(particles_, layers_, PEs_, triggerModes_))
print(f"{len(combinations)} configurations")
print(f"---> Run time: {len(combinations)*50/3600:.2f} hours")

3 configurations
---> Run time: 0.04 hours


In [8]:
fileList_ = rd.get_file_list(defname) #[:2]
print(f"---> Got {len(fileList_)} files.")

---> Getting file list for nts.sgrant.CosmicCRYExtractedCatTriggered.MDC2020ae_best_v1_3.root.
Done!
---> Got 96 files.


In [9]:
import sys
from datetime import datetime

In [28]:
# Generate a timestamped log filename
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
logName = f"../Logs/output_{timestamp}.log"
# Original standard output state
print("Log file is", logName)

Log file is ../Logs/output_20240914_011236.log


In [30]:
original_stdout = sys.stdout

# Open the log file
with open(logName, "w") as log:
    # Redirect stdout to the file
    sys.stdout = log
    try:
        # Run the jobs
        # print("test")
        Multiprocess(processFunction
                     , fileList_=fileList_
                     , recon=recon
                     , particles_=particles_
                     , PEs_=PEs_
                     , layers_=layers_
                     , triggerModes_=triggerModes_
                     , quiet=quiet
                     , resub=False)
    finally:
        # Reset stdout to its original state
        sys.stdout = original_stdout





Traceback (most recent call last):
  File "/cvmfs/mu2e.opensciencegrid.org/packages/spack/v0.21.0-dev.fermi/NULL/bin/spack", line 55, in <module>
==> Error: Keyboard interrupt.
==> Error: Keyboard interrupt.
==> Error: Keyboard interrupt.
==> Error: Keyboard interrupt.

==> Error: Keyboard interrupt.


==> Error: Keyboard interrupt.
==> Error: Keyboard interrupt.

==> Error: Keyboard interrupt.
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/sgrant/CRVSim/Analyses/PyMacros/MeasureModuleEfficiency.py", line 1023, in <module>
  File "/home/sgrant/CRVSim/Analyses/PyMacros/MeasureModuleEfficiency.py", line 1023, in <module>
Traceback (most recent call last):
  File "/home/sgrant/CRVSim/Analyses/PyMacros/MeasureModuleEfficiency.py", line 1023, in <module>
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/sgrant/CRVSim/Analyses/PyMacros/MeasureModuleEfficiency.py", line 1023, in <module>
    sys.exit(main())
Traceback 

KeyboardInterrupt: 

In [23]:
# def processFunction(fileName, recon, particles_, PEs_, layers_, triggerModes_, quiet):
# processFunction(fileName=[fileList_[0]]
#              , recon=recon
#              , particles_=particles_
#              , PEs_=PEs_
#              , layers_=layers_
#              , triggerModes_=triggerModes_
#              , quiet=quiet)

In [24]:
 # Multiprocess(processFunction
 #                     , fileList_=[fileList_[0]]
 #                     , recon=recon
 #                     , particles_=particles_
 #                     , PEs_=PEs_
 #                     , layers_=layers_
 #                     , triggerModes_=triggerModes_
 #                     , quiet=quiet
 #                     , resub=False)


---> Starting multiprocessing...

[2]
[2]
crv_trigger

---> nts.sgrant.CosmicCRYExtractedCatTriggered.MDC2020ae_best_v1_3.001205_00000000.root processed successfully! (100.0% complete)

---> Multiprocessing completed!
